<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=0469ee0dace7874e5430f69e23b30ebcaafde2752d7185c864fb29b85414f991
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-11 11:04:10
-------------------
qualified stocks: 86
with latest results: 24
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  99.50 K
Current:  1.42 C
-------------------
Today PnL: -45.42 K (-0.32%)
Current PnL: -22.66 L (-14.85%)
CY Booked + Current PnL: -9.47 L (-6.21%)
-------------------
Total profit:  1.49 L
Total loss:  -24.15 L
-------------------
Total Booked + Current PnL: 17.51 L (13.94%)
Total Booked PnL: 40.17 L (31.98%)
Curr Year Booked PnL: 13.19 L (9.26%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.33 C
Est FTT PnL: 90.13 L (63.26%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 7.31%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,101.07,50.0,M-SC,2.35,86650.0,-14127.0,14219.0,0.12,-14.02,16.41,0.09,245.0,-0.99,0.61,13.84,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-10.45,46.0,X-MC,3.16,88545.0,-2133.0,17036.0,0.03,-2.35,19.24,16.44,101.0,-0.13,0.63,17.35,X40,ATH,PHARMA
42,ITC,452.00,-38.34,44.0,X-LC,0.83,198461.0,-1677.0,22565.0,0.07,-0.84,11.37,10.44,4.0,-0.07,1.40,4.68,X40,NTT,FMCG
34,ICICIGI,2252.93,-14.07,54.0,X-MC,6.59,185628.0,11308.0,23890.0,-1.45,6.49,12.87,20.19,91.0,0.47,1.31,22.58,X40,ATH,INSURANCE
51,NESTLEIND,1377.00,-4.02,58.0,X-LC,8.20,295425.0,29999.0,26795.0,-0.03,11.30,9.07,21.40,11.0,1.12,2.09,18.70,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDUSINDBK,1800.0,283.05,71.0,L-MC,7.38,47002.0,-30204.0,55599.0,3.16,-39.12,118.29,32.89,258.0,-0.54,0.33,29.44,XR,NTT,BANKS
13,BERGEPAINT,680.0,-17.69,56.0,X-MC,1.09,221414.0,-5925.0,53981.0,2.07,-2.61,24.38,21.14,106.0,-0.11,1.57,24.80,XY24,NTT,PAINTS
38,INDIGOPNTS,1408.0,122.10,76.0,M-SC,5.36,145700.0,-28859.0,28892.0,1.70,-16.53,19.83,0.02,221.0,-1.00,1.03,26.73,OX40N,NTT,PAINTS
67,SIS,528.0,2041.97,50.0,H-SC,2.16,86445.0,-24587.0,48193.0,1.27,-22.14,55.75,21.26,156.0,-0.51,0.61,16.80,OX40N,NTT,MISC
74,TATAMOTORS,1065.0,-53.17,24.0,X-LC,36.18,148993.0,-127530.0,237599.0,1.17,-46.12,159.47,39.81,54.0,-0.54,1.05,5.01,XY24,NTT,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VAIBHAVGBL,521.0,64.19,53.0,H-SC,5.51,142410.0,-40365.0,151439.0,-3.70,-22.08,106.34,60.77,125.0,-0.27,1.01,28.25,XR,NTT,JEWELLERY
83,VOLTAS,1530.0,-6.11,37.0,X-MC,4.03,196950.0,5208.0,32556.0,-2.07,2.72,16.53,19.69,99.0,0.16,1.39,9.70,XY25,NTT,AC
11,BANDHANBNK,400.0,-20.14,33.0,H-SC,1.27,200236.0,-78323.0,335756.0,-2.01,-28.12,167.68,92.42,151.0,-0.23,1.42,15.72,XY24,NTT,BANKS
70,SURYODAY,240.0,46.93,34.0,H-SC,13.90,135354.0,-43717.0,109447.0,-1.88,-24.41,80.86,36.71,135.0,-0.40,0.96,34.30,XR,NTT,BANKS
84,WHIRLPOOL,2270.0,-44.07,40.0,M-SC,8.45,116012.0,1214.0,92833.0,-1.88,1.06,80.02,81.92,223.0,0.01,0.82,37.34,XR,NTT,DURABLES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,101.07,50.0,M-SC,2.35,86650.0,-14127.0,14219.0,0.12,-14.02,16.41,0.09,245.0,-0.99,0.61,13.84,OX40N,NTT,DURABLES
38,INDIGOPNTS,1408.0,122.10,76.0,M-SC,5.36,145700.0,-28859.0,28892.0,1.70,-16.53,19.83,0.02,221.0,-1.00,1.03,26.73,OX40N,NTT,PAINTS
46,KANSAINER,340.0,-68.16,41.0,H-SC,2.28,219933.0,-49734.0,86060.0,-0.59,-18.44,39.13,13.47,138.0,-0.58,1.55,11.83,XY24,NTT,PAINTS
67,SIS,528.0,2041.97,50.0,H-SC,2.16,86445.0,-24587.0,48193.0,1.27,-22.14,55.75,21.26,156.0,-0.51,0.61,16.80,OX40N,NTT,MISC
17,CERA,9475.0,-25.06,40.0,H-SC,1.80,136678.0,-39225.0,81241.0,0.32,-22.30,59.44,23.89,149.0,-0.48,0.97,18.04,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAIL,228.00,55.57,66.0,M-MC,10.44,246091.0,21129.0,146523.0,-0.99,9.39,59.54,74.53,192.0,0.14,1.74,44.68,XY24,BTT,STEEL
32,HINDZINC,730.22,29.16,53.0,M-LC,9.16,210453.0,5377.0,107184.0,-0.17,2.62,50.93,54.89,52.0,0.05,1.49,27.35,X5K,ATH,METALS
84,WHIRLPOOL,2270.00,-44.07,40.0,M-SC,8.45,116012.0,1214.0,92833.0,-1.88,1.06,80.02,81.92,223.0,0.01,0.82,37.34,XR,NTT,DURABLES
37,INDIAMART,4810.62,-53.89,49.0,H-SC,5.06,128896.0,5560.0,126073.0,-0.81,4.51,97.81,106.72,119.0,0.04,0.91,28.94,AR,ATH,MISC
62,SAMMAANCAP,326.00,-192.05,53.0,M-SC,35.15,158868.0,9648.0,134529.0,-1.52,6.47,84.68,96.62,208.0,0.07,1.12,71.93,XY25,NTT,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HINDZINC,730.22,29.16,53.0,M-LC,9.16,210453.0,5377.0,107184.0,-0.17,2.62,50.93,54.89,52.0,0.05,1.49,27.35,X5K,ATH,METALS
37,INDIAMART,4810.62,-53.89,49.0,H-SC,5.06,128896.0,5560.0,126073.0,-0.81,4.51,97.81,106.72,119.0,0.04,0.91,28.94,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-44.07,40.0,M-SC,8.45,116012.0,1214.0,92833.0,-1.88,1.06,80.02,81.92,223.0,0.01,0.82,37.34,XR,NTT,DURABLES
85,WIPRO,420.00,-15.61,45.0,M-LC,5.70,148676.0,-2269.0,111730.0,-0.02,-1.50,75.15,72.51,53.0,-0.02,1.05,4.43,XR,NTT,IT
47,KPIGREEN,731.05,3.26,45.0,H-SC,9.81,123241.0,-2056.0,60980.0,0.98,-1.64,49.48,47.03,141.0,-0.03,0.87,54.31,MH,ATH,POWER


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,GILLETTE,11206.78,-18.82,17.0,X-MC,5.54,249600.0,-4296.0,86611.0,-1.84,-1.69,34.70,32.42,105.0,-0.05,1.76,13.15,X40,ATH,FMCG
74,TATAMOTORS,1065.00,-53.17,24.0,X-LC,36.18,148993.0,-127530.0,237599.0,1.17,-46.12,159.47,39.81,54.0,-0.54,1.05,5.01,XY24,NTT,AUTO
45,JSWINFRA,342.00,-24.78,27.0,X-MC,4.44,188932.0,-11415.0,41924.0,0.39,-5.70,22.19,15.23,178.0,-0.27,1.34,24.72,X40N,NTT,REALTY
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.49,86926.0,-41744.0,87039.0,-0.21,-32.44,100.13,35.20,219.0,-0.48,0.61,0.00,X40,NTT,FOOTWEAR
23,DMART,5391.45,-20.66,29.0,X-LC,6.77,202080.0,-8327.0,67495.0,0.56,-3.96,33.40,28.12,38.0,-0.12,1.43,18.73,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,PGHH,17907.65,-31.74,36.0,X-MC,0.65,197715.0,-3105.0,70901.0,-0.78,-1.55,35.86,33.76,80.0,-0.04,1.40,3.57,X40,ATH,FMCG
42,ITC,452.00,-38.34,44.0,X-LC,0.83,198461.0,-1677.0,22565.0,0.07,-0.84,11.37,10.44,4.0,-0.07,1.40,4.68,X40,NTT,FMCG
66,SIEMENS,4671.50,-5.61,38.0,H-LC,1.61,151835.0,-34260.0,81733.0,-0.58,-18.41,53.83,25.51,15.0,-0.42,1.07,11.89,AR,ATH,ELECTRICAL
29,HAVELLS,2069.16,-9.44,35.0,X-MC,3.33,302253.0,-25738.0,132266.0,-1.26,-7.85,43.76,32.48,92.0,-0.19,2.14,2.68,X40,ATH,ELECTRICAL
79,UNITDSPR,1644.00,-6.52,61.0,X-LC,3.60,245066.0,10922.0,37691.0,1.09,4.66,15.38,20.77,86.0,0.29,1.73,11.91,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.49,86926.0,-41744.0,87039.0,-0.21,-32.44,100.13,35.20,219.0,-0.48,0.61,0.00,X40,NTT,FOOTWEAR
54,QUESS,986.00,-51.52,32.0,X-SC,37.25,48246.0,-16760.0,167689.0,-1.61,-25.78,347.57,232.18,198.0,-0.10,0.34,0.00,XY24,NTT,MISC
20,COLPAL,3726.84,-7.61,38.0,X-MC,7.15,215700.0,-47665.0,156986.0,-0.65,-18.10,72.78,41.51,84.0,-0.30,1.52,1.22,XY25,ATH,FMCG
52,PAGEIND,51769.92,-29.24,37.0,X-MC,7.65,159960.0,-3780.0,47124.0,-0.52,-2.31,29.46,26.47,82.0,-0.08,1.13,1.35,X40,ATH,APPARELS
1,ABB,7934.00,-42.88,32.0,H-LC,4.99,238776.0,-22843.0,142048.0,-0.35,-8.73,59.49,45.57,7.0,-0.16,1.69,1.60,AR,NTT,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-51.52,32.0,X-SC,37.25,48246.0,-16760.0,167689.0,-1.61,-25.78,347.57,232.18,198.0,-0.10,0.34,0.00,XY24,NTT,MISC
58,RELAXO,1176.00,-40.94,39.0,X-SC,4.04,80473.0,-64687.0,140610.0,0.02,-44.56,174.73,52.31,136.0,-0.46,0.57,7.72,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-41.87,27.0,X-SC,8.49,86926.0,-41744.0,87039.0,-0.21,-32.44,100.13,35.20,219.0,-0.48,0.61,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.45,46.0,X-MC,3.16,88545.0,-2133.0,17036.0,0.03,-2.35,19.24,16.44,101.0,-0.13,0.63,17.35,X40,ATH,PHARMA
33,HONAUT,58357.33,-23.74,39.0,X-SC,0.88,107520.0,-20418.0,67555.0,-0.49,-15.96,62.83,36.84,143.0,-0.30,0.76,10.07,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.97,-28.69,51.0,X-LC,9.38,284547.0,-61409.0,128103.0,0.06,-17.75,45.02,19.28,1.0,-0.48,2.01,5.19,X40,ATH,IT
40,INFY,2275.00,-16.73,62.0,X-LC,4.71,323866.0,10978.0,160702.0,0.46,3.51,49.62,54.87,3.0,0.07,2.29,12.08,X40,BTT,IT
42,ITC,452.00,-38.34,44.0,X-LC,0.83,198461.0,-1677.0,22565.0,0.07,-0.84,11.37,10.44,4.0,-0.07,1.40,4.68,X40,NTT,FMCG
82,VBL,671.64,-17.78,49.0,X-LC,5.76,295872.0,-19970.0,132639.0,0.11,-6.32,44.83,35.67,5.0,-0.15,2.09,6.66,X40N,ATH,FMCG
1,ABB,7934.00,-42.88,32.0,H-LC,4.99,238776.0,-22843.0,142048.0,-0.35,-8.73,59.49,45.57,7.0,-0.16,1.69,1.60,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,42.0,L-SC,27.24,76800.0,-36749.0,76800.0,-1.57,-32.36,100.00,35.27,268.0,-0.48,0.54,87.97,XR,NTT,HOTELS
7,ASIANTILES,137.00,7444.44,62.0,L-SC,13.97,82751.0,-11059.0,87675.0,-1.58,-11.79,105.95,81.67,269.0,-0.13,0.58,60.25,XR,NTT,CERAMICS
50,MASFIN,398.61,-19.60,48.0,H-SC,7.92,91515.0,-6465.0,28068.0,0.02,-6.60,30.67,22.05,152.0,-0.23,0.65,33.02,XR,ATH,FINANCE
47,KPIGREEN,731.05,3.26,45.0,H-SC,9.81,123241.0,-2056.0,60980.0,0.98,-1.64,49.48,47.03,141.0,-0.03,0.87,54.31,MH,ATH,POWER
80,VAIBHAVGBL,521.00,64.19,53.0,H-SC,5.51,142410.0,-40365.0,151439.0,-3.70,-22.08,106.34,60.77,125.0,-0.27,1.01,28.25,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7444.44,62.0,L-SC,13.97,82751.0,-11059.0,87675.0,-1.58,-11.79,105.95,81.67,269.0,-0.13,0.58,60.25,XR,NTT,CERAMICS
80,VAIBHAVGBL,521.00,64.19,53.0,H-SC,5.51,142410.0,-40365.0,151439.0,-3.70,-22.08,106.34,60.77,125.0,-0.27,1.01,28.25,XR,NTT,JEWELLERY
38,INDIGOPNTS,1408.00,122.10,76.0,M-SC,5.36,145700.0,-28859.0,28892.0,1.70,-16.53,19.83,0.02,221.0,-1.00,1.03,26.73,OX40N,NTT,PAINTS
6,ASIANPAINT,3465.66,-7.96,72.0,X-LC,6.15,232329.0,-19439.0,72649.0,-0.39,-7.72,31.27,21.13,27.0,-0.27,1.64,24.97,X40,ATH,PAINTS
68,SONACOMS,806.63,-31.67,65.0,M-SC,6.10,85102.0,-16057.0,56057.0,0.12,-15.87,65.87,39.54,202.0,-0.29,0.60,20.00,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.82
1,25,44.76
2,50,76.11


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.51
LC    30.45
MC    26.01
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.78
X40      19.32
X40N     12.45
XY25     10.20
XR        9.68
AR        7.74
OX40N     6.63
X200      1.79
X5K       1.49
SR        1.02
MH        0.87
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.96
X-MC    21.62
X-LC    20.98
M-SC    12.54
X-SC     5.56
H-LC     4.55
M-LC     3.93
H-MC     2.32
M-MC     1.74
L-SC     1.45
L-LC     0.99
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.99,-4.67,38.52
IT,12.60,-17.97,81.11
FINANCE,12.13,-10.20,60.15
MISC,7.70,-19.05,79.10
PAINTS,5.79,-12.69,29.48
ELECTRICAL,5.66,-12.90,52.92
INSURANCE,4.47,0.17,35.01
AUTO,2.71,-49.49,111.87
BANKS,2.71,-39.79,130.90


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3316535.0,21
XR,1286677.0,13
AR,1192787.0,9
X40,957669.0,13
X40N,708408.0,9
OX40N,545953.0,9
XY25,489244.0,7
SR,276688.0,2
X5K,107184.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3442557.0,24
X-MC,1352305.0,15
M-SC,1268716.0,15
X-LC,1070271.0,12
X-SC,561985.0,6
H-LC,294338.0,3
M-LC,267312.0,3
H-MC,259442.0,2
L-SC,254842.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1193329.0      6
           AR         844070.0      5
           XR         813112.0      7
M-SC       XY24       741275.0      6
X-MC       XY24       571230.0      4
X-LC       X40        442423.0      5
X-MC       X40        428207.0      7
X-LC       XY24       312040.0      2
X-SC       X40N       307257.0      4
H-SC       SR         276688.0      2
           OX40N      254378.0      3
X-LC       X40N       237825.0      3
H-LC       AR         223781.0      2
X-MC       XY25       189542.0      2
H-MC       XY24       184449.0      1
X-SC       XY24       167689.0      1
L-SC       XR         164475.0      2
X-MC       X40N       163326.0      2
M-MC       XY24       146523.0      1
M-SC       XR         141761.0      2
           XY25       134529.0      1
           OX40N      126215.0      4
           AR         124936.0      2
M-LC       XR         111730.0      1
           X5K        107184.0      1
L-SC       OX40N       90367.0      1
X-SC       X40         87039.0      1
X-LC       XY25        77983.0      2
H-MC       OX40N       74993.0      1
H-LC       X200        70557.0      1
H-SC       MH          60980.0      1
L-MC       XR          55599.0      1
M-LC       XY25        48398.0      1
L-LC       XY25        38792.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 50.0 seconds
